Importando a biblioteca geopy para o cálcuo da distância entre dois endereços e importa a biblioteca requests para as requisições na Api do Graphhopper.

In [3]:
from geopy.geocoders import Nominatim
from geopy import distance
import requests

# Inicializa o geocoder Nominatim com um nome de usuário para identificação
geocoder = Nominatim(user_agent="Liga jovem app")

Usando dois ceps *cep1* e *cep2* para a realização da distâncias, o programa utiliza também a Api do *_ViaCep_* para buscar o endereço completo dos dois ceps e transforma no formato .json().

In [4]:

# Definição de dois CEPs que serão usados para obter os endereços completos
cep1 = "49020-240"
cep2 = " 49107-553"

# Remove hifens e espaços dos CEPs para garantir que a consulta seja válida
cep1 = cep1.replace("-", "").replace(" ", "")
cep2 = cep2.replace("-", "").replace(" ", "")

# URLs da API ViaCEP para obter informações detalhadas dos endereços usando os CEPs
link1 = f"https://viacep.com.br/ws/{cep1}/json/"
link2 = f"https://viacep.com.br/ws/{cep2}/json/"


# Requisições HTTP para obter os dados dos endereços
response1 = requests.get(link1)
response2 = requests.get(link2)

# Conversão das respostas das APIs para JSON, facilitando o acesso aos dados
endereco1 = response1.json()
endereco2 = response2.json()

In [5]:
# Extração das informações necessárias dos endereços (logradouro, bairro, cidade)
log1 = endereco1["logradouro"]
bairro1 = endereco1["bairro"]
cidade1 = endereco1["localidade"]

log2 = endereco2["logradouro"]
bairro2 = endereco2["bairro"]
cidade2 = endereco2["localidade"]

Utilizando a *"geocodificação"* dos dois endereços e assim retornando o endereço em uma forma compativel com o geopy.

In [6]:
# Geocodificação dos endereços completos para obter as coordenadas (latitude e longitude)

endereco1_s = geocoder.geocode(f"{log1}, {cidade1}, Sergipe, Brazil")
endereco2_s = geocoder.geocode(f"{log2}, {cidade2}, Sergipe, Brazil")

# Impressão dos endereços geocodificados (pontos A e B)
print(endereco1_s)
print(endereco2_s)

Avenida Anísio Azevedo, 13 de Julho, Aracaju, Região Geográfica Imediata de Aracaju, Região Geográfica Intermediária de Aracaju, Sergipe, Região Nordeste, 49020-150, Brasil
Avenida Senador José Eduardo Dutra, Marcelo Deda, São Cristóvão, Região Geográfica Imediata de Aracaju, Região Geográfica Intermediária de Aracaju, Sergipe, Região Nordeste, 49096-291, Brasil


Calculando a latitudo e a longitude dos dois endereços para o cálculo da distância.

In [7]:
# Extração das coordenadas (latitude e longitude) dos endereços geocodificados
lat1, long1 = (endereco1_s.latitude), (endereco1_s.longitude)
lat2, long2 = (endereco2_s.latitude), (endereco2_s.longitude)

# Impressão das coordenadas para confirmação
print(lat1, long1)
print(lat2, long2)

-10.931144 -37.0473773
-10.9433459 -37.1020404


Utiliza a API do GraphHopper para calcular a distância real e o tempo estimado de viagem entre dois endereços, considerando as estradas e utilizando um carro como meio de transporte.

In [8]:

# API key do GraphHopper para calcular a rota entre os pontos usando um carro
api_key = "622dc0f7-87be-4a86-ac94-a21dff125961"

# URL para a API do GraphHopper, configurada para retornar a rota em português brasileiro
url = f"https://graphhopper.com/api/1/route?point={lat1},{long1}&point={lat2},{long2}&vehicle=car&locale=pt-BR&calc_points=true&key={api_key}"

# Requisição HTTP para obter os dados da rota entre os pontos A e B
response = requests.get(url)
dados = response.json()


In [9]:
# Extração dos dados da rota, incluindo a distância em quilômetros e o tempo em minutos
caminho = dados["paths"][0]
distancia_km = caminho["distance"]/1000
duracao_min = caminho["time"]/60000

# Impressão da distância e tempo estimado de viagem entre os dois pontos
print(f"{endereco1_s}\naté \n{endereco2_s}\n é {distancia_km:.2f} km em {duracao_min:.0f} mins")

Avenida Anísio Azevedo, 13 de Julho, Aracaju, Região Geográfica Imediata de Aracaju, Região Geográfica Intermediária de Aracaju, Sergipe, Região Nordeste, 49020-150, Brasil
até 
Avenida Senador José Eduardo Dutra, Marcelo Deda, São Cristóvão, Região Geográfica Imediata de Aracaju, Região Geográfica Intermediária de Aracaju, Sergipe, Região Nordeste, 49096-291, Brasil
 é 8.76 km em 19 mins


Geopy pode calcular a distância geodésica entre dois pontos usando a distância geodésica ou a distância do grande círculo.

A distância do grande círculo (great_circle) usa um modelo esférico da Terra, usando o raio médio da Terra conforme definido pela União Internacional de Geodésia e Geofísica, (2a + b)/3 = 6371,0087714150598 quilômetros aprox. 6371,009 km (para WGS-84), resultando em um erro de até cerca de 0,5%.

A distância geodésica é a menor distância na superfície de um modelo elipsoidal da Terra. O algoritmo padrão usa o método dado por Karney (2013) (geodésico).

In [10]:
# Cálculo a distância Geodésica entre os dois pontos e impressão do resultado

lugar1 = (lat1, long1)
lugar2 = (lat2, long2)
print(distance.distance(lugar1, lugar2))

6.1257974605192 km
